## Import packages

In [12]:
import os
import sys
import cv2
import json
import tqdm
import numpy as np
import pandas as pd
from time import time
from numpy.lib.stride_tricks import as_strided

import tensorflow as tf
import lucid.optvis.render as render
import lucid.modelzoo.vision_models as models
from keras.applications.inception_v3 import preprocess_input

sys.path.insert(0, '..')
from InceptionV1 import InceptionV1

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "7"

## Load model

In [13]:
model_wrapper = InceptionV1()
# model_wrapper.load_model()

## Load A-matrix

In [16]:
dir_path = '../../../data/InceptionV1/summit/A-mat'

In [24]:
A = {}
for blk in model_wrapper.BLKS:
    if '3x3' in blk:
        continue
    if '5x5' in blk: 
        continue
        
    file_path = '{}/A-0.03-{}.csv'.format(dir_path, blk)
    A[blk] = np.loadtxt(file_path, delimiter=',')

## Find important neurons per class

In [40]:
# Load class info
input_path = '../../../data/imagenet-labels.txt'
df = pd.read_csv(input_path, sep='\t')

# Parse class info
class_info = {}
for synset, tf_label in zip(df['synset'], df['tfrecord_label']):
    class_info[int(tf_label) - 1] = synset

In [83]:
important_neurons = {}
num_class = 1000

for c in range(num_class):
    
    important_neurons[class_info[c]] = {}
    
    for blk in model_wrapper.BLKS:

        if '3x3' in blk:
            continue
        if '5x5' in blk: 
            continue
    
        num_neurons_to_sample = int(A[blk].shape[-1] * 0.1)
        top_neurons = np.argsort(-A[blk][c])[:num_neurons_to_sample]
        top_cnts = A[blk][c][top_neurons]
        important_neurons[class_info[c]][blk] = [
            {'neuron': neuron, 'cnt': cnt}
            for neuron, cnt in zip(top_neurons, top_cnts)
        ]

In [84]:
# example: kit_fox
important_neurons['n02119789']['mixed4e'][:5]

[{'neuron': 36, 'cnt': 1041.0},
 {'neuron': 759, 'cnt': 836.0},
 {'neuron': 787, 'cnt': 800.0},
 {'neuron': 309, 'cnt': 305.0},
 {'neuron': 585, 'cnt': 218.0}]

## Find important groups (+ individual neurons) for each class

In [85]:
# Load group
group_dir_path = '../../../data/InceptionV1/bucket/bucket-all-10-20'

n2g = {}
g2n = {}

for blk in model_wrapper.BLKS:

    if '3x3' in blk:
        continue
    if '5x5' in blk: 
        continue
        
    bucket_path = '{}/buckets-{}.json'.format(group_dir_path, blk)
    with open(bucket_path, 'r') as f:
        bucket_data = json.load(f)
        
    for g in bucket_data:
        
        g2n['g-{}-{}'.format(blk, g)] = bucket_data[g]
        
        for n in bucket_data[g]:
            n2g[n] = 'g-{}-{}'.format(blk, g)
            

In [86]:
# Find important neurons and groups
num_class = 1000
important_groups = {}

for c in range(num_class):
    
    synset = class_info[c]
    important_groups[synset] = {}
    
    for blk in important_neurons[synset]:
        
        important_groups[synset][blk] = {}
        group_number = 0
        added_group = {}
        
        for top_neuron_info in important_neurons[synset][blk]:
            
            n = top_neuron_info['neuron']
            cnt = top_neuron_info['cnt']
            neuron = '{}-{}'.format(blk, n)
            
            if neuron in n2g:
                g = n2g[neuron]
                if g not in added_group:
                    
                    cnt = 0
                    for member_neuron in g2n[g]:
                        idx = int(member_neuron.split('-')[1])
                        cnt += A[blk][c, idx]
                    
                    important_groups[synset][blk][group_number] = {
                        'group': g2n[g],
                        'cnt': str(cnt / len(g2n[g]))
                    }
                    group_number += 1
                    added_group[g] = True
                    
                    
            else:
                important_groups[synset][blk][group_number] = {
                    'group': [neuron],
                    'cnt': str(cnt)
                }
                group_number += 1

In [95]:
# Sort important groups by cnt
for synset in important_groups:
    for blk in important_groups[synset]:
        sorted_nodes = sorted(important_groups[synset][blk].items(), key=lambda x: x[1]['cnt'], reverse=True)
        only_sorted_nodes = [item[1] for item in sorted_nodes]
        important_groups[synset][blk] = only_sorted_nodes

In [96]:
# Save important group
output_dir_path = '../../../data/InceptionV1/graph/node/'
for synset in important_groups:
    file_path = '{}/node-{}.json'.format(output_dir_path, synset)
    with open(file_path, 'w') as f:
        json.dump(important_groups[synset], f)